# 패키지 설치

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pycaret

     |████████████████████████████████| 264 kB 7.5 MB/s 
     |████████████████████████████████| 111 kB 51.1 MB/s 
     |████████████████████████████████| 1.7 MB 31.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.0 MB 21.9 MB/s 
     |████████████████████████████████| 14.6 MB 123 kB/s 
     |████████████████████████████████| 1.3 MB 43.5 MB/s 
     |████████████████████████████████| 6.8 MB 48.5 MB/s 
     |████████████████████████████████| 80 kB 10.2 MB/s 
     |████████████████████████████████| 248 kB 55.3 MB/s 
     |████████████████████████████████| 167 kB 59.3 MB/s 
     |████████████████████████████████| 271 kB 59.3 MB/s 
     |████████████████████████████████| 56 kB 5.7 MB/s 
     |████████████████████████████████| 636 kB 39.5 MB/s 
     |████████████████████████████████| 675 kB 44.6 MB/s 
     |█████████████████

In [ ]:
import pandas as pd 
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from pycaret.classification import *
from sklearn.metrics import log_loss

# 데이터 불러오기

In [ ]:
cd '/content/drive/MyDrive/DCC/data'

/content/drive/MyDrive/DCC/data


In [ ]:
train = pd.read_csv('./train.csv', index_col=0)
test = pd.read_csv("./test.csv", index_col=0)
submission = pd.read_csv('./sample_submission.csv', index_col=0)
train.fillna('NAN', inplace=True) 
test.fillna('NAN', inplace=True)

In [ ]:
print(train.shape)
print(test.shape)
print(submission.shape)

(26457, 19)
(10000, 18)
(10000, 3)


In [ ]:
# 타겟 voted
clf = setup(data = train, target="credit")

,Description,Value
0,session_id,3807
1,Target,credit
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(26457, 19)"
5,Missing Values,False
6,Numeric Features,5
7,Categorical Features,13
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
#logloss 적용 및 predict_proba로 제출하기 위해 metric 추가
add_metric('logloss', 'LogLoss', log_loss, greater_is_better=False, target="pred_proba")

Name                                                           LogLoss
Display Name                                                   LogLoss
Score Function                   <function log_loss at 0x7f7d8505f950>
Scorer               make_scorer(log_loss, greater_is_better=False,...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: logloss, dtype: object

# 모델비교

In [ ]:
#svm, ridge는 predict_proba 미지원으로 제외
best = compare_models(sort = 'logloss', n_select = 3, exclude=['svm','ridge'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.7003,0.7107,0.4297,0.6922,0.6261,0.2452,0.3245,0.7611,0.670
gbc,Gradient Boosting Classifier,0.6937,0.6502,0.4116,0.6378,0.6095,0.2140,0.3033,0.7947,8.211
lda,Linear Discriminant Analysis,0.6447,0.6056,0.3384,0.5499,0.5122,0.0147,0.0593,0.8626,0.216
nb,Naive Bayes,0.6429,0.6057,0.3337,0.4643,0.5036,0.0012,0.0130,0.8634,0.053
lr,Logistic Regression,0.6427,0.5370,0.3333,0.4131,0.5030,0.0000,0.0000,0.8812,0.590
rf,Random Forest Classifier,0.6994,0.7501,0.5361,0.6771,0.6824,0.3674,0.3741,1.0010,2.871
ada,Ada Boost Classifier,0.6920,0.6272,0.4057,0.6186,0.6035,0.2019,0.2998,1.0787,0.810
et,Extra Trees Classifier,0.6706,0.7086,0.5160,0.6491,0.6552,0.3120,0.3170,2.5405,2.589
knn,K Neighbors Classifier,0.6172,0.6587,0.4655,0.5961,0.6038,0.2143,0.2166,3.9981,0.391
dt,Decision Tree Classifier,0.6060,0.6353,0.4913,0.6154,0.6103,0.2529,0.2532,13.2365,0.232


# 모델 블렌딩

In [ ]:
blended = blend_models(estimator_list = best, fold = 5, optimize = 'logloss')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,0.6941,0.6706,0.4104,0.7399,0.6083,0.2114,0.3057,0.7949
1,0.6933,0.6887,0.4086,0.6766,0.6066,0.2087,0.3030,0.7901
2,0.6952,0.6906,0.4096,0.6256,0.6080,0.2103,0.3115,0.7886
3,0.6925,0.6840,0.4066,0.6243,0.6041,0.2016,0.3032,0.7921
4,0.6981,0.6745,0.4148,0.6244,0.6136,0.2238,0.3198,0.7931
Mean,0.6946,0.6817,0.4100,0.6581,0.6081,0.2112,0.3087,0.7918
SD,0.0019,0.0078,0.0027,0.0455,0.0031,0.0072,0.0063,0.0022
